In [1]:
import tensorflow as tf
import numpy as np
import cv2
import pandas as pd
import urllib.request


In [63]:
## Below code loads the original given CSV file (small-2oq-c1r.csv) and save a smaller CSV containing only "tops"
## Uncomment below code for first time generating the product listing of "tops" and saving in another CSV 

#df1 = pd.read_csv("small-2oq-c1r.csv", error_bad_lines=False, low_memory=False)
#print(df1)

#df2 = df1.set_index('categories', drop=False)
#df3 = df2.loc["Apparels>Women>Western Wear>Shirts, Tops & Tunics>Tops"]
#df3.to_csv('tops_listings_small.csv', index=False)

In [64]:
df3 = pd.read_csv("tops_listings_small.csv")

C:\Users\DELL\Anaconda3\envs\dog-project\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [143]:
## listings with index as productId
df3 = df3.set_index('productId', drop=False)
N = df3.shape[0]
print("Total number of products with sub-category tops = ", N)

n = (int)(N/1000.0)
print("Current samples this program is running for: ",n) 

Total number of products with sub-category tops =  333162
Current samples this program is running for:  333


In [66]:
## test code
prod = df3.loc['TOPE7BDHRF6CHJ7G']
prod.mrp

1549

In [4]:
def getImageUrls(prod):
    url_str = prod.imageUrlStr
    img_urls = url_str.split(';')
    return img_urls

In [5]:
def getImagesFromUrls(urls):
    img=[]
    for url in urls:
        req = urllib.request.urlopen(url)
        arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
        img.append(cv2.imdecode(arr, -1)) # 'Load it as it is'
        #print(url)
    return img

In [6]:
from scipy import spatial
from gensim import models

model = models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
index2word_set = set(model.wv.index2word)

def avg_feature_vector(sentence, model, num_features, index2word_set):
    words = sentence.split()
    feature_vec = np.zeros((num_features, ), dtype='float32')
    n_words = 0
    for word in words:
        if word in index2word_set:
            n_words += 1
            feature_vec = np.add(feature_vec, model[word])
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec

C:\Users\DELL\Anaconda3\envs\dog-project\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\DELL\Anaconda3\envs\dog-project\lib\site-packages\ipykernel\__main__.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


In [7]:
from PIL import Image
import imagehash
import sys
import time

def similarImagesHash(products, n):
    similarImages = {}
    for i in range(0, n):
        urls = getImageUrls(products.iloc[i])
        #print("total images in this listing are :", len(urls))
        #urls[0]
    
        image = getImagesFromUrls(urls)
        #print(len(image))
        #hashc = hashp = None
        hash = str(imagehash.average_hash(Image.fromarray(image[0])))
        
## below for loop is to check if all images are similar for a given product

#        for j in range (len(image)):
#            #print(image[j].shape)
#            hashc = imagehash.average_hash(Image.fromarray(image[j]))
#            if (hashc != hashp):
#                print("product ",i,": different hashc and hashp, images are not similar -", hashc, hashp)
#            
#            hashPrev = hashc

        #if hash0 in similarImages:
            #print(products.iloc[i].productId, '(for index', i,')  already exists as', ' '.join(similarImages[hash0]))
        
        if(i%5 == 0):
            print("Progress",i,"/",n, end="\r")
            #time.sleep(1)
            #sys.stdout.flush()
        similarImages[hash] = similarImages.get(hash, []) + [products.iloc[i].productId]

    return similarImages

In [8]:
similarImages = similarImagesHash(df3, n)

In [112]:
## display the dictionary with all tuples with similar images (only primary image is taken into consideration)
#for k, v in similarImages.items():
#    print(k, v)

In [11]:
## writing similar image products disctionary to a JSON file for easy access
import json
json.dump(similarImages, open('similarImages.json', 'w'))

In [12]:
## reading the similar Image products from JSON 
simIm = json.load(open('similarImages.json', 'r'))

#for keys, values in simIm.items():
#    print(keys, values)

In [35]:
## below is the test code
urls = getImageUrls(df3.iloc[0])

print("total images in this listing are :", len(urls))
print(urls[0])

image = getImagesFromUrls(urls)
print(len(image))
for i in range (len(image)):
    print(image[i].shape)


total images in this listing are : 4
http://img.fkcdn.com/image/top/r/h/n/1-1-wwtpws0128-citrine-l-original-imae9a7e9txtzwz2.jpeg
4
(1498, 667, 3)
(200, 89, 3)
(400, 178, 3)
(800, 356, 3)


In [15]:
## display one product with all the fields
df3.iloc[0]
#df3.iloc[53].description

productId                                               TOPE9ABBZU3HZRHN
title                  Citrine Casual Short Sleeve Printed Women's Pi...
description            This beautiful printed modal top from Citrine ...
imageUrlStr            http://img.fkcdn.com/image/top/r/h/n/1-1-wwtpw...
mrp                                                                 1099
sellingPrice                                                         329
specialPrice                                                         329
productUrl             http://dl.flipkart.com/dl/citrine-casual-short...
categories             Apparels>Women>Western Wear>Shirts, Tops & Tun...
productBrand                                                     Citrine
productFamily          TOPE9ABBBTJYDSQE,TOPE9ABBHJ8HGGGK,TOPE9ABBPDAN...
inStock                                                            False
codAvailable                                                        True
offers                                             

In [16]:
def areImagesSameForProducts(prod1, prod2):
    if(prod1.imageUrlStr == prod2.imageUrlStr):
        return True
    return False

In [28]:
def similarText(text1, text2):
    s1_afv = avg_feature_vector(text1, model=model, num_features=300, index2word_set=index2word_set)
    s2_afv = avg_feature_vector(text2, model=model, num_features=300, index2word_set=index2word_set)
    sim = 1 - spatial.distance.cosine(s1_afv, s2_afv)
    #print(sim)
    return sim


In [67]:
def titleDescSimilarity(prod1, prod2, str):
    if(str == "title"):
        text1 = prod1.title
        text2 = prod2.title
    elif(str == "desc"):
        text1 = prod1.description
        text2 = prod2.description
    else:
        return -1

    if(text1 and text2):
        try:
            simText = similarText(text1, text2)
        except:
            #print("AttributeError ---:")
            return -1

        simText = round(simText,4)

## below commented code is for debug and prints

#        if (simText<=1.0 and simText >0.99):
#            print(simText)
#            print(df3.iloc[i].productId, text1)
#            print(df3.iloc[j].productId, text2)
#            urlsi = getImageUrls(df3.iloc[i])
#            print(urlsi[0])
#            urlsj = getImageUrls(df3.iloc[j])
#            print(urlsj[0])

    return simText

In [74]:
## test code
titleDescSimilarity(df3.iloc[0], df3.iloc[1], "desc")

-1

In [146]:
## The dictionary JSON preparation for duplicate detected products
## considering products with same imagehash as duplicate with probablity 1.0 (100%)
## Assumptions:
## products with only difference in sizes are considered duplicates 
## products with only difference in color are not considered duplicates 

duplicate_produts = {}
for keys, values in simIm.items():
    #print("------")
    #print(keys)
    #print("*****")
    key = values[0]
    if (len(values) == 1):
        duplicate_produts[key] = duplicate_produts.get(key, []) + []
    #print(key)
    for i in range(1, len(values)):
        #print(len(values), values[0])
        duplicate_produts[key] = duplicate_produts.get(key, []) + [[values[i], str(1.0)]]
        #print(values[i])


In [147]:
#json.dump(duplicate_produts, open('dedup_products.json', 'w'))
#dedup = json.load(open('dedup_products.json', 'r'))


In [148]:
def deDuplicateFunc(prod1, prod2):
    key = prod1.productId
    try:
        if(prod2.productId in duplicate_produts[key]):
            return
        else:
            val = prod2.productId
    except:
        key = prod2.productId
        try:
            if(prod2.productId in duplicate_produts[key]):
                return
            else:
                val = prod1.productId
        except:
            return
        
    titleSim = titleDescSimilarity(prod1, prod2, "title")
    DescSim = titleDescSimilarity(prod1, prod2, "desc")
    if(titleSim==1.0 and DescSim == 1.0):
            duplicate_produts[key] = duplicate_produts.get(key, []) + [[val, str(0.6)]]
    if(titleSim<1.0 and titleSim > 0.99):
        if(DescSim<1.0 and DescSim>0.99):
            if(prod2.productId in duplicate_produts[key]):
                duplicate_produts[key] = duplicate_produts.get(key, []) + [[val, str(0.5)]]


In [149]:
#df3.shape[0]
#count = 0
for i in range(0, n):
    if(i%10 == 0):
        print("Progress",i,"/",n, end="\r")
    for j in range(i+1, n):
        deDuplicateFunc(df3.iloc[i], df3.iloc[j])


In [150]:
json.dump(duplicate_produts, open('dedup_products.json', 'w'))